# *A Bayesian CNN with uncertainty measures*



**Source**:
*https://github.com/kumar-shridhar/PyTorch-Softplus-Normalization-Uncertainty-Estimation-Bayesian-CNN/blob/master/main.ipynb*

**Paper**:
*https://arxiv.org/abs/1806.05978*


*Aleatoric* = uncertainty due to inherent noisiness of **data**

*Epistemic* = uncertainty due to the **model**.





In [1]:
!git clone https://github.com/kumar-shridhar/PyTorch-Softplus-Normalization-Uncertainty-Estimation-Bayesian-CNN.git

Cloning into 'PyTorch-Softplus-Normalization-Uncertainty-Estimation-Bayesian-CNN'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [2]:
!git clone https://github.com/kumar-shridhar/PyTorch-BayesianCNN.git 
!ls 

Cloning into 'PyTorch-BayesianCNN'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1214 (delta 118), reused 186 (delta 84), pack-reused 991
Receiving objects: 100% (1214/1214), 67.72 MiB | 32.09 MiB/s, done.
Resolving deltas: 100% (719/719), done.
PyTorch-BayesianCNN						    sample_data
PyTorch-Softplus-Normalization-Uncertainty-Estimation-Bayesian-CNN


In [3]:
#change working directory to access inner modules
%cd PyTorch-BayesianCNN
%cd Image Recognition
#check working directory
!pwd

/content/PyTorch-BayesianCNN
[Errno 2] No such file or directory: 'Image Recognition'
/content/PyTorch-BayesianCNN
/content/PyTorch-BayesianCNN


In [34]:
from __future__ import print_function

import os
import sys
import time
import argparse
import datetime
import math
import pickle


import torchvision
import torchvision.transforms as transforms

import torch
import torch.utils.data as data
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from collections import OrderedDict
from torchvision import datasets

time: 7.91 ms


In [30]:
net_type = 'alexnet'
#dataset = 'CIFAR10'
dataset = 'MNIST'
outputs = 10
inputs = 3
resume = False
n_epochs = 20
lr = 0.01
weight_decay = 0.0005
num_samples = 1
beta_type = "Blundell"
resize=32

time: 2.19 ms


In [31]:
# Hyper Parameter settings
use_cuda = torch.cuda.is_available()
torch.cuda.set_device(0)

time: 1.28 ms


In [32]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load... maybe this should be smaller (32)
batch_size = 32
# percentage of training set to use as validation
valid_size = 0.2

time: 1.08 ms


In [0]:
# convert data to a normalized torch.FloatTensor
transform_cifar = transforms.Compose([
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# choose the training and test datasets
if dataset == 'CIFAR10':
  train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_cifar)
  test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_cifar)
  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           num_workers=num_workers)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers)
  
elif dataset == 'MNIST':
  train_data = datasets.MNIST('data', train=True,
                              download=True, transform=transforms.Compose([transforms.ToTensor(),]))
  test_data = datasets.MNIST('data', train=False,
                              download=True, transform=transforms.Compose([transforms.ToTensor(),]))
  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           num_workers=num_workers)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers)
  
  # train_loader = torch.utils.data.DataLoader(
  #       datasets.MNIST('mnist-data/', train=True, download=True,
  #                      transform=transforms.Compose([transforms.ToTensor(),])),
  #                      batch_size=batch_size, shuffle=True)#, worker_init_fn=np.random.seed(12))

  # test_loader = torch.utils.data.DataLoader(
  #       datasets.MNIST('mnist-data/', train=False, transform=transforms.Compose([transforms.ToTensor(),])),
  #                      batch_size=batch_size, shuffle=True)#, worker_init_fn=np.random.seed(12))

In [73]:

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
#classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


time: 1.39 ms


In [89]:
class BBBConv2d(nn.Module):
    def __init__(self, q_logvar_init, p_logvar_init, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False):
        super(BBBConv2d, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.q_logvar_init = q_logvar_init
        self.p_logvar_init = p_logvar_init
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.bias = bias

        self.mu_weight = nn.Parameter(torch.Tensor(out_channels, in_channels // groups, kernel_size, kernel_size))
        self.sigma_weight = nn.Parameter(torch.Tensor(out_channels, in_channels // groups, kernel_size, kernel_size))
        self.register_buffer('eps_weight', torch.Tensor(out_channels, in_channels // groups, kernel_size, kernel_size))
        
        self.reset_parameters()

    def reset_parameters(self):
        n = self.in_channels
        n *= self.kernel_size ** 2
        stdv = 1.0 / math.sqrt(n)
        self.mu_weight.data.uniform_(-stdv, stdv)
        self.sigma_weight.data.fill_(self.p_logvar_init)

    def forward(self, input):
        raise NotImplementedError()


    def convprobforward(self, input):
        sig_weight = torch.exp(self.sigma_weight)
        weight = self.mu_weight + sig_weight * self.eps_weight.normal_()
        kl_ = math.log(self.q_logvar_init) - self.sigma_weight + (sig_weight**2 + self.mu_weight**2) / (2 * self.q_logvar_init ** 2) - 0.5
        bias = None
        
        out = F.conv2d(input, weight, bias, self.stride, self.padding, self.dilation, self.groups)
        kl = kl_.sum() 
        return out, kl

time: 25.8 ms


In [90]:

class BBBLinearFactorial(nn.Module):
    def __init__(self, q_logvar_init, p_logvar_init, in_features, out_features, bias=False):
        super(BBBLinearFactorial, self).__init__()
        self.q_logvar_init = q_logvar_init
        self.in_features = in_features
        self.out_features = out_features
        self.p_logvar_init = p_logvar_init
        self.mu_weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.sigma_weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.register_buffer('eps_weight', torch.Tensor(out_features, in_features))
        
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.mu_weight.size(1))
        self.mu_weight.data.uniform_(-stdv, stdv)
        self.sigma_weight.data.fill_(self.p_logvar_init)
        self.eps_weight.data.zero_()

    def forward(self, input):
        raise NotImplementedError()
        

    def fcprobforward(self, input):
        sig_weight = torch.exp(self.sigma_weight)
        weight = self.mu_weight + sig_weight * self.eps_weight.normal_()
        kl_ = math.log(self.q_logvar_init) - self.sigma_weight + (sig_weight**2 + self.mu_weight**2) / (2 * self.q_logvar_init ** 2) - 0.5
        bias = None
        out = F.linear(input, weight, bias)
        kl = kl_.sum() 
        return out, kl

time: 29.8 ms


In [91]:
class BBBAlexNet(nn.Module):
    '''The architecture of AlexNet with Bayesian Layers'''

    def __init__(self, outputs, inputs):
        super(BBBAlexNet, self).__init__()

        self.q_logvar_init = 0.05
        self.p_logvar_init = math.log(0.05)
 
        self.classifier = BBBLinearFactorial(self.q_logvar_init, self.p_logvar_init, 1* 1 * 128, outputs)

        self.conv1 = BBBConv2d(self.q_logvar_init, self.p_logvar_init, inputs, 64, kernel_size=11, stride=4, padding=5)
        self.soft1 = nn.Softplus()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = BBBConv2d(self.q_logvar_init,  self.p_logvar_init, 64, 192, kernel_size=5, padding=2)
        self.soft2 = nn.Softplus()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = BBBConv2d(self.q_logvar_init, self.p_logvar_init, 192, 384, kernel_size=3, padding=1)
        self.soft3 = nn.Softplus()

        self.conv4 = BBBConv2d(self.q_logvar_init, self.p_logvar_init, 384, 256, kernel_size=3, padding=1)
        self.soft4 = nn.Softplus()

        self.conv5 = BBBConv2d(self.q_logvar_init, self.p_logvar_init, 256, 128, kernel_size=3, padding=1)
        self.soft5 = nn.Softplus()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.flatten = FlattenLayer(1 * 1 * 128)
        # self.fc1 = BBBLinearFactorial(q_logvar_init, N, p_logvar_init, 1* 1 * 128, outputs)


        layers = [self.conv1, self.soft1, self.pool1, self.conv2, self.soft2, self.pool2, self.conv3, self.soft3,
                  self.conv4, self.soft4, self.conv5, self.soft5, self.pool3]

        self.layers = nn.ModuleList(layers)

    def probforward(self, x):
        kl = 0
        for layer in self.layers:
            if hasattr(layer, 'convprobforward') and callable(layer.convprobforward):
                x, _kl, = layer.convprobforward(x)
            else:
                x = layer.forward(x)
        x = x.view(x.size(0), -1)
        x, _kl = self.classifier.fcprobforward(x)
        kl += _kl
        logits = x
        return logits, kl

time: 37.3 ms


In [92]:
# Architecture
net_type = 'alexnet'
if (net_type == 'lenet'):
    net = BBBLeNet(outputs,inputs)
elif (net_type == 'alexnet'):
    net = BBBAlexNet(outputs, inputs)
elif (net_type == '3conv3fc'):
    net = BBB3Conv3FC(outputs,inputs)
else:
    print('Error : Network should be either [LeNet / AlexNet / 3Conv3FC]')

time: 26.9 ms


In [93]:
#check if cuda is available
if use_cuda:
    net.cuda()

time: 15.1 ms


In [94]:
# create checkpoint path to save
ckpt_name = f'model_{net_type}_{dataset}_bayesian.pt'
ckpt_name

'model_alexnet_MNIST_bayesian.pt'

time: 4.04 ms


In [95]:
def get_beta(batch_idx, m, beta_type):
    if beta_type == "Blundell":
        beta = 2 ** (m - (batch_idx + 1)) / (2 ** m - 1)
    elif beta_type == "Soenderby":
        beta = min(epoch / (num_epochs // 4), 1)
    elif beta_type == "Standard":
        beta = 1 / m
    else:
        beta = 0
    return beta

time: 6.82 ms


In [96]:

def elbo(out, y, kl, beta):
    loss = F.cross_entropy(out, y)
    return loss + beta * kl

time: 1.32 ms


In [97]:

def get_beta(epoch_idx, N):
    return 1.0 / N / 100

time: 1.47 ms


**Uncertainty estimation**

In [98]:

def calc_uncertainty_softmax(output):
    prediction = F.softmax(output, dim = 1)
    results = torch.max(prediction, 1 )
    p_hat = np.array(results[0].cpu().detach())
    epistemic = np.mean(p_hat ** 2, axis=0) - np.mean(p_hat, axis=0) ** 2
    #epistemic += epistemic 
    #print (epistemic)
    aleatoric = np.mean(p_hat * (1-p_hat), axis = 0)
    #aleatoric += aleatoric
    #print (aleatoric)
    return epistemic, aleatoric

time: 8.19 ms


But we will report **normalized** uncertainty measures:

In [99]:
def normalization_function(x):
    return (x) / torch.sum(x, dim=0)

time: 1.54 ms


In [100]:
def calc_uncertainty_normalized(output):
    outputs = []
    for t in range(1):
        prediction = F.softplus(output.cpu())
        prediction = normalization_function(prediction)
        outputs.append(prediction)
        
    res = np.mean(prediction.numpy(), axis=0)
    p_hat= torch.cat(outputs, 1)
    p_hat=p_hat.numpy()
    T=1
    
    aleatoric = np.diag(res) - p_hat.T.dot(p_hat)/p_hat.shape[0]
#     aleatoric += aleatoric
    tmp = p_hat - res  
    epistemic = tmp.T.dot(tmp)/tmp.shape[0]
#     epistemic += epistemic 

    #print(np.sum(epistemic, keepdims = True))
    #print(np.sum(aleatoric, keepdims = True))
    #p_hat = np.array(res)
    #print (p_hat)
    #epistemic = np.mean((p_hat) ** 2, axis=0) - np.mean((p_hat), axis=0) ** 2
    #epistemic = np.mean((1-p_hat) ** 2, axis=0) - np.mean((1-p_hat), axis=0) ** 2
    #epistemic += epistemic 
    #print (epistemic)
    #aleatoric = np.mean((p_hat) * (1-(p_hat)), axis = 0)
    #aleatoric = np.mean((1-p_hat), axis = 0) ** 2
    #aleatoric += aleatoric
    #print (aleatoric)
    #print((np.sum(epistemic, keepdims = True)), (np.sum(aleatoric, keepdims = True)))
    return (np.sum(epistemic, keepdims = True)), (np.sum(aleatoric, keepdims = True))

time: 15.1 ms


# Bayesian training

In [101]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs, kl = net.probforward(inputs)
        loss = elbo(outputs, targets, kl, get_beta(epoch, len(train_data)))
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')
    #with open( writefile_train, 'a' ) as f:
    #        writer = csv.writer(f)
    #        writer.writerow([epoch, 100.*correct/total])

time: 13.5 ms


In [102]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    accuracy_max = 0    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs, _ = net.probforward(inputs)
            epistemic , aleatoric = calc_uncertainty_softmax(outputs)
            norm_epistemic , norm_aleatoric = calc_uncertainty_normalized(outputs)
            #print(epistemic)
            #print(norm_epistemic[0][0])
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            accuracy = 100.*correct/total
        print(f'[TEST] Acc: {accuracy:.3f}')
        print(f'Epistemic Uncertainty: {epistemic:.6f}')
        print(f'Aleatoric Uncertainty: {aleatoric:.6f}\n')
        #unlike authors originally do, report both raw and normalized uncertainties
        print(f'Normalized Epistemic Uncertainty: {norm_epistemic[0][0]:.6f}')
        print(f'Normalized Aleatoric Uncertainty: {norm_aleatoric[0][0]:.6f}\n')
        #print(map('Normalized Epistemic Uncertainty:{.6f}'.format,norm_epistemic[0]))
        #with open( writefile_test, 'a' ) as f:
        #    writer = csv.writer(f)
        #    writer.writerow([epoch, 100.*correct/total, epistemic, aleatoric])
        

    torch.save(net.state_dict(), ckpt_name)

time: 16.7 ms


This has to be changed, we are not using the same number of epochs

In [103]:
# train for how many epochs?
epochs = 20
count = 0

time: 1.35 ms


**Bayesian training and reporting accuracy + uncertainties:**

In [104]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.4 s


In [105]:
from torch.optim import Adam
import random


torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
random.seed(12)

torch.backends.cudnn.deterministic=True


optimizer = Adam(net.parameters(), lr=lr)
for _ in range(epochs):
    train(count)
    test(count)
    count += 1
lr /= 10

Epoch: 0


RuntimeError: ignored

time: 42.9 ms


# Frequentist AlexNet on MNIST

CIFAR10 later

In [58]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform(m.weight, gain=np.sqrt(2))
        nn.init.normal_(m.weight, mean=0, std=1)
        nn.init.constant(m.bias, 0)

class AlexNet(nn.Module):

    def __init__(self, num_classes, inputs=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(inputs, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


time: 18.6 ms


In [77]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform(m.weight, gain=np.sqrt(2))
        nn.init.normal_(m.weight, mean=0, std=1)
        nn.init.constant(m.bias, 0)

class LeNet(nn.Module):
    def __init__(self, num_classes, inputs=3):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(inputs, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)

        return(out)


time: 18.7 ms


In [78]:
dataset = 'MNIST'
outputs = 10
inputs = 3
n_epochs = 100
lr = 0.001
resize=32

time: 4.74 ms


In [79]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 32 
# percentage of training set to use as validation
valid_size = 0.2

time: 2.03 ms


In [80]:

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

time: 9.79 ms


In [81]:
from torch.utils.data.sampler import SubsetRandomSampler
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

time: 1.29 ms


In [82]:
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

time: 2.09 ms


In [83]:
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

time: 1.19 ms


In [84]:
# Architecture
net_type = "alexnet"
if (net_type == 'lenet'):
    net = LeNet(outputs,inputs)
elif (net_type == 'alexnet'):
    net = AlexNet(outputs,inputs)
elif (net_type == '3conv3fc'):
        net = ThreeConvThreeFC(outputs,inputs)
else:
    print('Error : Network should be either [LeNet / AlexNet / 3Conv3FC')

#print(net)

time: 26.3 ms


In [85]:
# move tensors to GPU if CUDA is available
if use_cuda:
    net.cuda()

time: 7.37 ms


In [86]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(net.parameters(), lr=lr)

time: 2.03 ms


In [87]:
ckpt_name = f'model_{net_type}_{dataset}_frequentist.pt'
ckpt_name

'model_alexnet_MNIST_frequentist.pt'

time: 4.48 ms


In [88]:
%%time
import random
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
random.seed(12)

torch.backends.cudnn.deterministic=True



correct = 0
total = 0
val_correct = 0
val_total = 0

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    net.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = net(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        pred = torch.max(output, dim=1)[1]
        correct += torch.sum(pred.eq(target)).item()
        total += target.numel()
    ######################    
    # validate the model #
    ######################
    net.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = net(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        #validation accuracy
        _, predicted = output.max(1)
        val_correct += predicted.eq(target).sum().item()
        val_total += target.size(0)
        
        

    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print('Training Accuracy: {:.3f} \tValidation Accuracy: {:.3f}'.format(
        100.*correct/total, 100.*val_correct/val_total))
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(net.state_dict(), ckpt_name)
        valid_loss_min = valid_loss

RuntimeError: ignored

time: 82.2 ms


In [0]:
net.load_state_dict(torch.load(ckpt_name))

<All keys matched successfully>

In [0]:
%%time

# track test loss# track  
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

net.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = net(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not use_cuda else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        if i >= target.data.shape[0]: # batch_size could be greater than left number of images
            break
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.137909

Test Accuracy of airplane: 76% (767/1000)
Test Accuracy of automobile: 73% (736/1000)
Test Accuracy of  bird: 31% (312/1000)
Test Accuracy of   cat: 41% (418/1000)
Test Accuracy of  deer: 64% (645/1000)
Test Accuracy of   dog: 54% (545/1000)
Test Accuracy of  frog: 77% (776/1000)
Test Accuracy of horse: 70% (705/1000)
Test Accuracy of  ship: 73% (739/1000)
Test Accuracy of truck: 69% (693/1000)

Test Accuracy (Overall): 63% (6336/10000)
CPU times: user 1.92 s, sys: 129 ms, total: 2.05 s
Wall time: 2.05 s
